<a href="https://colab.research.google.com/github/claudia-viaro/optimal_stopping-switching/blob/main/opt_switching_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Suppose we consider both possible final regimes of the process

In [10]:
import numpy as np
import torch
import torch.nn as nn
np.random.seed(234198)
import itertools
import random
import time
import scipy.stats
import math
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.utils.data as tdata

In [11]:
class BlackScholes:
  def __init__(self, drift, sigma, delta, spot, assets,  paths, periods,
         maturity, strike, dividend=0):

    self.drift = drift - dividend
    self.sigma = sigma
    self.delta = delta
    self.spot = spot
    self.assets = assets
    self.paths = paths
    self.periods = periods
    self.maturity = maturity
    self.strike = strike
    self.dt = self.maturity / self.periods
    self.df = math.exp(-self.drift * self.dt)

  def drift_fct(self, x, t):
    del t
    return self.drift * x

  def diffusion_fct(self, x, t, v=0):
    del t
    return self.sigma * x



  def simulate_process(self):
    """Returns a nparray (nb_paths * assets * nb_dates) with prices."""
    paths = self.paths
    spot_paths = np.empty((self.periods+1, paths, self.assets ))

    spot_paths[0, :, :] = self.spot
    random_numbers = np.random.normal(
        0, 1, (self.periods, paths, self.assets ))
    dW = random_numbers * np.sqrt(self.dt)
    drift = self.drift
    r = np.repeat(np.repeat(np.repeat(
        np.reshape(drift, (-1, 1, 1)), self.periods, axis=0),
        paths, axis=1), self.assets, axis=2)
    sig = np.ones((self.periods, paths, self.assets))*self.sigma
    #sig = np.repeat(np.repeat(np.repeat(
    #    np.reshape(self.sigma, (-1, 1, 1)), self.periods+1, axis=2),
    #    paths, axis=1), self.assets, axis=0)
    
    spot_paths[1:, :,  :] = np.repeat(
        spot_paths[0:1, :, :], self.periods, axis=0)* np.exp(np.cumsum((r-self.delta) * self.dt - (sig ** 2) * self.dt / 2 + sig * dW, axis=0))

    return spot_paths #.reshape(spot_paths.shape[2], spot_paths.shape[0], spot_paths.shape[1])


'''
PLOT
'''

def draw_stock_model(stockmodel):
    stock_paths = stockmodel

    # draw a path
    one_path = stock_paths[:, 0, 0]
    dates = np.array([i for i in range(len(one_path))])
    plt.plot(dates, one_path, label='stock path')
    plt.ylabel('Stock price')
    plt.ylabel('Time')
    plt.legend()
    return plt.show()



class Ftheta_NN(nn.Module):
  def __init__(self, assets):
    super(Ftheta_NN, self).__init__()
    H = assets + 40
    self.bn0 = nn.BatchNorm1d(num_features=assets)
    self.layer1 = nn.Linear(assets, H)
    self.leakyReLU = nn.LeakyReLU(0.5)
    self.Softplus = nn.Softplus()
    self.sigmoid = nn.Sigmoid()
    self.tanh = nn.Tanh()
    self.relu = nn.ReLU()
    self.bn1 = nn.BatchNorm1d(num_features=H)
    self.layer2 = nn.Linear(H, H)
    self.bn2 = nn.BatchNorm1d(num_features=H)
    self.layer3 = nn.Linear(H, 1)
    self.bn3 = nn.BatchNorm1d(num_features=1)

  def forward(self, x):
    x = self.bn0(x)
    x = self.layer1(x)
    x = self.relu(x)
    x = self.bn2(x)
    x = self.layer3(x)
    x = self.sigmoid(x)
    return x



'''
class Ftheta_NN(nn.Module):
  def __init__(self, assets, hidden_size):
    super(Ftheta_NN, self).__init__()
    self.l1 = nn.Linear(assets, hidden_size) 
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size, hidden_size)
    self.l3 = nn.Linear(hidden_size, 1)  
    self.sigmoid=nn.Sigmoid()
    
  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    out = self.relu(out)
    out = self.l3(out)
    out = self.sigmoid(out)
    return out
'''
# set initial weights of a linear layer of the NN with uniform values and bias=0.01 (or choose zero initial weights)
def init_weights(m):
  if isinstance(m, torch.nn.Linear):
    torch.manual_seed(42)
    # torch.nn.init.zeros_(m.weight)
    torch.nn.init.xavier_uniform_(m.weight)
    m.bias.data.fill_(0.01)


In [12]:
class Profit:
  def __init__(self, model):
    self.strike = model.strike
    self.model = model
    
  def terminal_(self, X):
    terminal = np.max(X, axis=1) - self.strike
    return terminal.clip(0, None)

  def g(self, date,path,X):
    X=torch.from_numpy(X).float()
    max1=torch.max(X[int(date) , path , : ].float()-self.strike)
    return np.exp(-self.model.drift*self.model.dt*date)*torch.max(max1,torch.tensor([0.0])) 


  def terminal(self, X):
    payoff = np.max(X) - self.strike
    return payoff.clip(0, None)

  def running(self, Y, X, switch_to):
    
    r_benefit = self.terminal(X)
    for i in range(0, self.model.paths):
      if switch_to[i] == 0:
        gamma = -self.terminal(X[i]) # there are two rows, the first for \gamma_{0,1}, the second for \gamma_{1,0}
      else:
        gamma: self.terminal(X[i]) + 0.7  

    return torch.from_numpy(r_benefit+Y-gamma)  

  def current_payoff(self, X, Y, regime_path):
  # X is stock_paths[date, :, :]
  # Y is Y_train_i[date+1, :] or Y_train_j
  # regime_path is regime_path_i[date+1, :]
  
    current_payoff = np.zeros((self.model.paths))
    
    for i in range(0, self.model.paths):
      if int(regime_path[i]) == 0:
        gamma = -self.terminal(X[i, :])
      else:
        gamma= self.terminal(X[i, :]) + 0.7  
      current_payoff[i]=self.terminal(X[i, :])-gamma+Y[i]    

    return current_payoff   

  

In [13]:
class Optimization(object):

  def __init__(self, assets, paths, epochs=50, batch_size=2000):
    self.assets = assets
    self.paths = paths
    self.epochs = epochs
    self.batch_size = batch_size
    self.network = Ftheta_NN(self.assets).double()
    self.network.apply(init_weights)


  def train_network(self,  stock_values, current_payoff,
                    future_payoff):
    optimizer = optim.Adam(self.network.parameters())
    future_payoff = torch.from_numpy(future_payoff).double()
    current_payoff = torch.from_numpy(current_payoff).double()
    X_inputs = torch.from_numpy(stock_values).double()

    self.network.train(True)
    ones = torch.ones(self.paths)
    for epoch in range(self.epochs):
      optimizer.zero_grad()
      outputs = self.network(X_inputs).reshape(-1) # probabilities
      reward = (current_payoff * outputs ) +future_payoff * (ones - outputs) # reward function
      loss = -torch.mean(reward) # loss function
      loss.backward() # gradient calculation of the loss function
      optimizer.step() # gradient descent update

  def evaluate_network(self, X_inputs):
    self.network.train(False)
    X_inputs = torch.from_numpy(X_inputs).double()
    outputs = self.network(X_inputs)
    return outputs.view(X_inputs.size()[0]).detach().numpy(), self.network

In [14]:
# i=0, j=1

class Training_large:
  def __init__(self, model, payoff, nb_epochs=50):

    self.model = model # argument is S    
    self.neural_stopping = Optimization(self.model.assets, self.model.paths) 
    self.payoff_function = payoff(self.model)

  def price(self):
    model = self.model
    stock_paths = self.model.simulate_process()    
    disc_factor = np.math.exp((-model.drift) * model.maturity/(model.periods))
    
    # create empty objects to store values
    models = [None]*self.model.periods, [None]*self.model.periods

    regimes = [0, 1]
    regime_path_i=np.zeros((self.model.periods+1, self.model.paths)) # record at which regime we're at at each n
    regime_path_j=np.zeros((self.model.periods+1, self.model.paths))
    regimes_path=np.array([regime_path_i, regime_path_j]) #you can call each regime by regimes_path[0]

    Y_train_i=np.zeros((self.model.periods+1, self.model.paths))
    Y_train_j=np.zeros((self.model.periods+1, self.model.paths))
    Y_train = np.array([Y_train_i, Y_train_j])

    F_theta_i=np.zeros((self.model.periods+1,self.model.paths))
    F_theta_j=np.zeros((self.model.periods+1,self.model.paths))
    F_theta=np.array([F_theta_i, F_theta_j])
    F_theta[1][self.model.periods, 0:self.model.paths] = 1

    values = np.array([np.zeros(self.model.paths), np.zeros(self.model.paths)])

    # at n=N
    final_payoff = np.array([self.payoff_function.terminal(stock_paths[-1, :, :])])
    future_payoff = final_payoff*disc_factor


    for r in regimes:
      # still at maturity
      Y_train[r][self.model.periods, :]=final_payoff
      regimes_path[r][self.model.periods, :] = regimes[r]
      values[r] = Y_train[r][self.model.periods, :]
      print("final regime", r, "date", self.model.periods, ":", 1.00," , ", 1.00, " , ", self.model.paths, "value", round(np.mean(values[r]),3))

      for date in range(stock_paths.shape[0] - 2, 0, -1):
        current_payoff = np.array([np.zeros(self.model.paths), np.zeros(self.model.paths)])
        current_payoff[r] = self.payoff_function.current_payoff(stock_paths[date, :, :], Y_train[r][date+1, :], regimes_path[r][date+1, :])
        stopping_probability=np.array([np.zeros(self.model.paths), np.zeros(self.model.paths)])
        Nnetworks=np.array([np.zeros(self.model.paths), np.zeros(self.model.paths)])
        stopping_probability[r], models[r][date] = self.stop(stock_paths[date, : , :], 
                                    current_payoff[r],
                                    future_payoff)
        
        F_theta[r][date,:]=(stopping_probability[r] > 0.5)*1.00   # transform stopping probabilities in 0-1 decision
        #which_i = stopping_probability_i > 0.5

        for m in range(0,self.model.paths-1):
          old_regime = regimes_path[r][date +1, m]
          regimes_path[r][date, m] = int(F_theta[r][date,m])   #int(which_i[m])   #current regime using probabilities just obtained

          if int(old_regime) - regimes_path[r][date, m]>0:  #gamma 0-1
            gamma = self.payoff_function.g(date, m, stock_paths)+0.7
          elif int(old_regime) - regimes_path[r][date, m]<0: 
            gamma = - self.payoff_function.g(date, m, stock_paths) #gamma 1-0  
          else:
            gamma = 0 
          Y_train[r][date, m] = Y_train[r][date+1, m]- gamma

        immediate_exercise_value = Y_train[r][date, :]       
        values[r][int(regimes_path[r][date, m])] = immediate_exercise_value[int(regimes_path[r][date, m])] # when we switch we take the current profit
        values[r][~int(regimes_path[r][date, m])] *= disc_factor           # when we don't switch we take final profit discounted 
        print("final regime", r, "date", date, ":", round(np.min(stopping_probability[r]), 3)," , ", round(np.max(stopping_probability[r]), 3), " , ", len([1 for l in stopping_probability[r] if l > 0.5]), "value", round(np.mean(values[r]),3))          


    return models

  def stop(self, stock_values, current_payoff,
           future_payoff):
     
    self.neural_stopping.train_network(
      stock_values,
      current_payoff ,
      future_payoff)

    inputs = stock_values
    stopping_probability , networks   = self.neural_stopping.evaluate_network(inputs)
    return stopping_probability , networks  

In [15]:
hyperparam_training = {'drift': 0.2, 'sigma': 0.05, 'delta': 0.1,  'paths':1024, 'periods': 9, 'maturity': 3., 'strike' : 100,'assets':2,  'spot':90,}
S_training=BlackScholes(**hyperparam_training)


Price_training = Training_large(S_training, Profit, nb_epochs=3000)

'''
arguments are:
- path process
- payoff class of functions
- number of epochs to be used for the gradient descent algorithm
'''

Models = Price_training.price()

final regime 0 date 9 : 1.0  ,  1.0  ,  1024 value 60.301
final regime 0 date 8 : 0.0  ,  0.96  ,  759 value 60.297
final regime 0 date 7 : 0.0  ,  0.986  ,  871 value 60.299
final regime 0 date 6 : 0.0  ,  0.988  ,  943 value 60.295
final regime 0 date 5 : 0.0  ,  0.994  ,  991 value 60.291
final regime 0 date 4 : 0.0  ,  0.998  ,  999 value 60.288
final regime 0 date 3 : 0.0  ,  0.999  ,  1016 value 60.285
final regime 0 date 2 : 0.997  ,  1.0  ,  1024 value 60.282
final regime 0 date 1 : 0.999  ,  1.0  ,  1024 value 60.279
final regime 1 date 9 : 1.0  ,  1.0  ,  1024 value 60.301
final regime 1 date 8 : 1.0  ,  1.0  ,  1024 value 60.297
final regime 1 date 7 : 1.0  ,  1.0  ,  1024 value 60.293
final regime 1 date 6 : 1.0  ,  1.0  ,  1024 value 60.29
final regime 1 date 5 : 0.0  ,  1.0  ,  1023 value 60.287
final regime 1 date 4 : 1.0  ,  1.0  ,  1024 value 60.284
final regime 1 date 3 : 1.0  ,  1.0  ,  1024 value 60.281
final regime 1 date 2 : 1.0  ,  1.0  ,  1024 value 60.279
final

In [16]:
# testing bound

class Testing_large:
  def __init__(self, model, payoff, models):   
    self.model = model # argument is S   
    self.neural_stopping = Optimization(model.assets, model.paths) 
    self.payoff_function = payoff(self.model)
    self.models = models

  def price(self):
    model = self.model
    disc_factor = np.math.exp((-model.drift) * model.maturity/(model.periods))
    stock_paths = self.model.simulate_process()


    # create empty objects to store values

    regimes = [0, 1]
    regime_path_i=np.zeros((self.model.periods+1, self.model.paths)) # record at which regime we're at at each n
    regime_path_j=np.zeros((self.model.periods+1, self.model.paths))
    regimes_path=np.array([regime_path_i, regime_path_j]) #you can call each regime by regimes_path[0]

    Y_train_i=np.zeros((self.model.periods+1, self.model.paths))
    Y_train_j=np.zeros((self.model.periods+1, self.model.paths))
    Y_train = np.array([Y_train_i, Y_train_j])

    F_theta_i=np.zeros((self.model.periods+1,self.model.paths))
    F_theta_j=np.zeros((self.model.periods+1,self.model.paths))
    F_theta=np.array([F_theta_i, F_theta_j])
    F_theta[1][self.model.periods, 0:self.model.paths] = 1

    values = np.array([np.zeros(self.model.paths), np.zeros(self.model.paths)])

    # at n=N
    final_payoff = np.array([self.payoff_function.terminal(stock_paths[-1, :, :])])
    future_payoff = final_payoff*disc_factor

    for r in regimes:
      # still at maturity
      Y_train[r][self.model.periods, :]=final_payoff
      regimes_path[r][self.model.periods, :] = regimes[r]
      values[r] = Y_train[r][self.model.periods, :]
      print("final regime", r, "date",  self.model.periods, ":", round(np.mean(values[r]),3))

      for date in range(stock_paths.shape[0] - 2, 0, -1):
        current_payoff = np.array([np.zeros(self.model.paths), np.zeros(self.model.paths)])
        current_payoff[r] = self.payoff_function.current_payoff(stock_paths[date, :, :], Y_train[r][date+1, :], regimes_path[r][date+1, :])
        current_model=self.models[r][date]
        probs=current_model(torch.from_numpy(stock_paths[date])) 
        np_probs=probs.detach().numpy().reshape(self.model.paths)
      
        F_theta[r][date,:]=(np_probs > 0.5)*1.0   # transform stopping probabilities in 0-1 decision
        #which = np_probs > 0.5        
        

        for m in range(0,self.model.paths-1):
          old_regime = regimes_path[r][date +1, m]
          regimes_path[r][date, m] = int(F_theta[r][date,m])   #int(which_i[m])   #current regime using probabilities just obtained

          if int(old_regime) - regimes_path[r][date, m]>0:  #gamma 0-1
            gamma = self.payoff_function.g(date, m, stock_paths)+0.7
          elif int(old_regime) - regimes_path[r][date, m]<0: 
            gamma = - self.payoff_function.g(date, m, stock_paths) #gamma 1-0  
          else:
            gamma = 0 
          Y_train[r][date, m] = Y_train[r][date+1, m]- gamma

        immediate_exercise_value = Y_train[r][date, :]       
        values[r][int(regimes_path[r][date, m])] = immediate_exercise_value[int(regimes_path[r][date, m])] # when we switch we take the current profit
        values[r][~int(regimes_path[r][date, m])] *= disc_factor           # when we don't switch we take final profit discounted 
        print("final regime", r, "date", date, ":", round(np.mean(values[r]),3))          


    return round(np.mean(values)* disc_factor, 3), Y_train

In [17]:
hyperparam_testing = {'drift': 0.2, 'sigma': 0.05, 'delta': 0.1,  'paths':50000, 'periods': 9, 'maturity': 3., 'strike' : 100,'assets':2,  'spot':90,}
S_testing=BlackScholes(**hyperparam_testing)

In [18]:
price_testing = Testing_large(S_testing, Profit, Models)

Y_test_mean, Y_train = price_testing.price()
print(Y_test_mean)

final regime 0 date 9 : 76.327
final regime 0 date 8 : 76.327
final regime 0 date 7 : 76.327
final regime 0 date 6 : 76.327
final regime 0 date 5 : 76.327
final regime 0 date 4 : 76.327
final regime 0 date 3 : 76.327
final regime 0 date 2 : 76.327
final regime 0 date 1 : 76.327
final regime 1 date 9 : 76.327
final regime 1 date 8 : 76.327
final regime 1 date 7 : 76.327
final regime 1 date 6 : 76.327
final regime 1 date 5 : 76.327
final regime 1 date 4 : 76.327
final regime 1 date 3 : 76.327
final regime 1 date 2 : 76.327
final regime 1 date 1 : 76.327
71.404
